In [89]:
import pandas as pd
import numpy as np
import plotly.express as px
import os

### Read in csv files

In [90]:
folder_path = "./outputs_combined/scenarios/combined/"

figure_outputs = './report_figures/'
if not os.path.exists(figure_outputs):
    os.makedirs(figure_outputs)

output_files = [
    "book_val",
    "consumption_costs",
    "consumption_emissions",
    "energy_consumption",
    "fuel_type",
    "is_retrofit_vec_table",
    "methane_leaks",
    "operating_costs",
    "peak_consump",
    "retrofit_cost",
    "retrofit_year",
    "stranded_val"
]
df_dic = {}

for f in output_files:
    filepath = os.path.join(folder_path, f"{f}.csv")
    df = pd.read_csv(filepath)
    df_dic[f] = df

In [91]:
df_dic['book_val']

,year,book_val,asset_id,existing_or_retrofit,asset_domain,asset_type,scenario
0,2020,0.00000,F_714592_2938902,retrofit,building,building_aggregate,accelerated_elec
1,2021,0.00000,F_714592_2938902,retrofit,building,building_aggregate,accelerated_elec
2,2022,0.00000,F_714592_2938902,retrofit,building,building_aggregate,accelerated_elec
3,2023,0.00000,F_714592_2938902,retrofit,building,building_aggregate,accelerated_elec
4,2024,0.00000,F_714592_2938902,retrofit,building,building_aggregate,accelerated_elec
...,...,...,...,...,...,...,...
15451,2061,48238.71500,GP001,retrofit,gas_network,gas_main,hybrid_gas_immediate
15452,2062,36179.03625,GP001,retrofit,gas_network,gas_main,hybrid_gas_immediate
15453,2063,24119.35750,GP001,retrofit,gas_network,gas_main,hybrid_gas_immediate
15454,2064,12059.67875,GP001,retrofit,gas_network,gas_main,hybrid_gas_immediate


In [92]:
def change_scenario_names(df):
    # udpate scenario names for report
    scenario_name_map = {"continued_gas": 'Continued Gas',
                        "hybrid_gas" : 'Dual Fuel - Pipeline (EOL)',
                        "hybrid_gas_immediate" : 'Dual Fuel - Pipeline (Accelerated)',
                        "natural_elec" : 'Unmanaged Electrification',
                        "hybrid_npa" : 'Dual Fuel (Tank)',
                        "accelerated_elec" : 'Accelerated Electrification',}

    return df.replace({'scenario': scenario_name_map},inplace=True)

def filter_data(df, column, scenarios): # filter dataframe for specific scenarios
    return df[df[column].isin(scenarios)]

def filter_years(df, year_range): # filter dataframe for specific years, inclusive
    return df[df.year.between(year_range[0],year_range[1])]

In [93]:
# update scenario names for each dataframe
for key in df_dic:
    change_scenario_names(df_dic[key])

### Energy Consumption

In [94]:
plot_data = 'energy_consumption'

groupby_columns = ['year','energy_type','scenario'] # sum rows to get yearly consumption by scenario and energy type
scenarios = ['Continued Gas',
            'Dual Fuel - Pipeline (EOL)', 
            'Dual Fuel - Pipeline (Accelerated)',
            'Unmanaged Electrification',
            'Dual Fuel (Tank)',
            'Accelerated Electrification',] 
year_range = [2020, 2065]

# group, filter data based on what will be shown in report
energy_consump = df_dic[plot_data].groupby(by=groupby_columns).sum().reset_index()
energy_consump = filter_data(energy_consump, 'scenario', scenarios)
if 'year' in groupby_columns:
    energy_consump = filter_years(energy_consump, year_range)

# convert consumption to MkWh
energy_consump['consumption'] = energy_consump['consumption']/1000

# table = pd.pivot_table(energy_consump, 'consumption', index=['scenario'], columns='energy_type', aggfunc=np.sum)

In [95]:
# plot
fig = px.bar(energy_consump, x="year", y="consumption", color="energy_type",
             facet_col="scenario", facet_col_spacing=0.01, 
             barmode="stack",
             width = 1300,
             category_orders={"scenario": scenarios,
                            'energy_type':['electricity',
                                           'natural_gas', 
                                           'propane',
                                           'fuel_oil', ],
                            },
              color_discrete_map={'electricity' : 'cornflowerblue',
                                'natural_gas' : 'palevioletred', 
                                'propane' : 'lightseagreen',
                                'fuel_oil' : 'goldenrod'},
              labels={'year':'', # remove 'year' from xaxis on each subplot
                    } 
            )

# update names for plotting
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
trace_dic = {'electricity' : 'Electricity',
        'natural_gas' : 'Pipeline Gas', 
        'propane' : 'LPG',
        'fuel_oil' : 'Fuel Oil',}
fig.for_each_trace(lambda t: t.update(name = trace_dic[t.name]))

# general formatting
fig.update_layout(bargap=0,
                  legend=dict(title_text='', x=0.92, y=0.98), # set legend location, properties
                  yaxis_title='Energy Consumption (MWh)',
                  plot_bgcolor='#f5f5f5',
                  )
fig.update_traces(marker_line_width=0)
fig.update_xaxes(tickangle= -90)
# fig.update_yaxes(showgrid=False)

fig.show()
fig.write_html(figure_outputs+'energy_consump.html')

### Peak Consumption

In [96]:
plot_data = 'peak_consump'

groupby_columns = ['year','energy_type','scenario', 'asset_id'] # sum rows to get yearly peak consumption by scenario, energy type, and asset id
scenarios = ['Continued Gas',
            'Dual Fuel - Pipeline (EOL)', 
            'Dual Fuel - Pipeline (Accelerated)',
            'Unmanaged Electrification',
            'Dual Fuel (Tank)',
            'Accelerated Electrification',] 
year_range = [2020, 2065]

# group, filter data based on what will be shown in report
peak_consump = df_dic[plot_data].groupby(by=groupby_columns).sum().reset_index()
peak_consump = filter_data(peak_consump, 'scenario', scenarios)
if 'year' in groupby_columns:
    peak_consump = filter_years(peak_consump,year_range)

xfmr_capacity = 75 # temp placeholder - to be updated
peak_consump['over_capacity'] = [True if i > xfmr_capacity else False for i in peak_consump['peak_consump']]

peak_consump

,year,energy_type,scenario,asset_id,peak_consump,over_capacity
0,2020,electricity,Accelerated Electrification,TB001,23.675806,False
1,2020,electricity,Accelerated Electrification,TB002,22.052421,False
2,2020,electricity,Accelerated Electrification,TB003,18.963890,False
3,2020,electricity,Accelerated Electrification,TB004,21.247716,False
4,2020,electricity,Continued Gas,TB001,23.675806,False
...,...,...,...,...,...,...
1099,2065,electricity,Dual Fuel - Pipeline (EOL),TB004,26.229684,False
1100,2065,electricity,Unmanaged Electrification,TB001,79.829321,True
1101,2065,electricity,Unmanaged Electrification,TB002,74.385738,False
1102,2065,electricity,Unmanaged Electrification,TB003,63.943553,False


In [97]:
fig = px.bar(peak_consump, x="year", y="peak_consump", color="over_capacity",
             facet_col="scenario", facet_col_spacing=0.01, facet_row='asset_id',
             barmode="stack",
             width = 1300,
             category_orders={"scenario": scenarios,
                            },
              color_discrete_map={True : 'red',
                                False : 'green'},
              labels={
                    'year':'', # remove 'year' from xaxis on each subplot
                    } 
            )

# update names for plotting
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
trace_dic = {'True' : 'Overloaded',
            'False' : 'Sufficient Capacity',}
fig.for_each_trace(lambda t: t.update(name = trace_dic[t.name]))

# general formatting
fig.update_layout(bargap=0,
                  legend=dict(title_text='', x=0.01, y=0.995), # set legend location, properties
                  yaxis_title='kW',
                  plot_bgcolor='#f5f5f5',
                  )
fig.update_xaxes(tickangle= -90)
# TODO: update peak_consump column name to kW
# fig.update_yaxes(title_text='kW')

fig.show()
fig.write_html(figure_outputs+'peak_consump.html')

### Retrofit Costs

In [98]:
plot_data = 'retrofit_cost'

groupby_columns = ['scenario', 'asset_id', 'asset_domain'] # sum rows to get yearly cost by scenario and asset domain
rfit_scenarios = ['Continued Gas',
                    'Dual Fuel - Pipeline (EOL)', 
                    'Dual Fuel - Pipeline (Accelerated)',
                    'Dual Fuel (Tank)',
                    'Accelerated Electrification',]
year_range = [2020, 2065]

# group, filter data based on what will be shown in report
rfit_costs_byBuilding = df_dic[plot_data].groupby(by=groupby_columns).sum().reset_index()
rfit_costs_byBuilding = filter_data(rfit_costs_byBuilding, 'scenario', rfit_scenarios)
if 'year' in groupby_columns:
    rfit_costs_byBuilding = filter_years(rfit_costs_byBuilding, year_range)
rfit_costs_byBuilding = filter_data(rfit_costs_byBuilding, 'asset_domain', ['building']) # only looking at building assets

rfit_costs_byBuilding

,scenario,asset_id,asset_domain,year,retrofit_cost
0,Accelerated Electrification,F_714592_2938902,building,93955,36315.598968
1,Accelerated Electrification,F_714723_2938816,building,93955,41196.334765
2,Accelerated Electrification,F_714815_2939003,building,93955,29406.073680
3,Accelerated Electrification,F_714897_2938859,building,93955,36500.249160
4,Accelerated Electrification,F_714956_2939056,building,93955,32558.922648
...,...,...,...,...,...
246,Dual Fuel - Pipeline (EOL),F_715419_2938797,building,93955,41392.135489
247,Dual Fuel - Pipeline (EOL),F_715497_2938593,building,93955,28856.523508
248,Dual Fuel - Pipeline (EOL),F_715583_2938646,building,93955,47145.889020
249,Dual Fuel - Pipeline (EOL),F_715583_2938872,building,93955,36074.141014


In [99]:
fig = px.box(rfit_costs_byBuilding, x="scenario", y="retrofit_cost",
             height=800, width=800, 
             category_orders={"scenario": rfit_scenarios,}
            )

fig.update_layout(bargap=0,
                  yaxis_title='Cost of Building Intervention by Building ($)',
                  plot_bgcolor='#f5f5f5',
                  )
fig.update_xaxes(tickangle= -90)
# fig.update_yaxes(showgrid=False)

fig.show()
fig.write_html(figure_outputs+'cost_by_building.html')

In [109]:
# running sum of retrofit costs by asset domain
plot_data = 'retrofit_cost'

groupby_columns = ['year','scenario', 'asset_domain'] # sum rows to get yearly cost by scenario and asset domain
scenarios = ['Continued Gas',
            'Dual Fuel - Pipeline (EOL)', 
            'Dual Fuel - Pipeline (Accelerated)',
            'Unmanaged Electrification',
            'Dual Fuel (Tank)',
            'Accelerated Electrification',]
year_range = [2020, 2065]

# group, filter data based on what will be shown in report
rfit_costs_byFuel = df_dic[plot_data].groupby(by=groupby_columns).sum().reset_index()
rfit_costs_byFuel['cum_cost'] = rfit_costs_byFuel.groupby(['scenario', 'asset_domain'])['retrofit_cost'].cumsum() # calcuate running sum of stranded value costs by scenario and asset domain
rfit_costs_byFuel = filter_data(rfit_costs_byFuel, 'scenario', scenarios)
if 'year' in groupby_columns:
    rfit_costs_byFuel = filter_years(rfit_costs_byFuel, year_range)


rfit_costs_byFuel

,year,scenario,asset_domain,retrofit_cost,cum_cost
0,2020,Accelerated Electrification,building,0.0,0.00000
1,2020,Accelerated Electrification,elec_network,0.0,0.00000
2,2020,Accelerated Electrification,gas_network,0.0,0.00000
3,2020,Continued Gas,building,0.0,0.00000
4,2020,Continued Gas,elec_network,0.0,0.00000
...,...,...,...,...,...
823,2065,Dual Fuel - Pipeline (EOL),elec_network,0.0,0.00000
824,2065,Dual Fuel - Pipeline (EOL),gas_network,0.0,655687.15000
825,2065,Unmanaged Electrification,building,0.0,892384.30557
826,2065,Unmanaged Electrification,elec_network,0.0,80000.00000


In [110]:
fig = px.bar(rfit_costs_byFuel, x="year", y="cum_cost", color="asset_domain",
             facet_col="scenario", facet_col_spacing=0.01, 
             barmode="stack",
             width = 1300,
             category_orders={"scenario": scenarios,
                              'asset_domain': ['gas_network','elec_network','building',]
                            },
              color_discrete_map={'building' : 'cornflowerblue',
                                'elec_network' : 'goldenrod', 
                                'gas_network' : 'palevioletred'},
              labels={'year':'', # remove 'year' from xaxis on each subplot
                    } 
            )

# update names for plotting
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
trace_dic = {'building' : 'Building',
            'elec_network' : 'Electric', 
            'gas_network' : 'Gas'}
fig.for_each_trace(lambda t: t.update(name = trace_dic[t.name]))

# general formatting
fig.update_layout(bargap=0,
                  legend=dict(title_text='', x=0.92, y=0.98), # set legend location, properties
                  yaxis_title='Running Sum of Retrofit Costs ($)',
                  plot_bgcolor='#f5f5f5',
                  )
fig.update_traces(marker_line_width=0)
fig.update_xaxes(tickangle= -90)
# fig.update_yaxes(showgrid=False)

fig.show()
fig.write_html(figure_outputs+'running_retrofit_cost.html')

### Stranded Asset Value

In [113]:
plot_data = 'stranded_val'

groupby_columns = ['year','scenario', 'asset_type'] # sum rows to get yearly cost by scenario and asset type
sv_scenarios = ['Unmanaged Electrification',
                'Accelerated Electrification',]
year_range = [2025, 2050]

# group, filter data based on what will be shown in report
stranded_val = df_dic[plot_data].groupby(by=groupby_columns).sum().reset_index()
stranded_val['cum_val'] = stranded_val.groupby(['scenario', 'asset_type'])['stranded_val'].cumsum() # calcuate running sum of stranded value costs
stranded_val = filter_data(stranded_val, 'scenario', sv_scenarios)
if 'year' in groupby_columns:
    stranded_val = filter_years(stranded_val, year_range)

stranded_val

,year,scenario,asset_type,stranded_val,cum_val
90,2025,Accelerated Electrification,building_aggregate,162853.993226,162853.993226
91,2025,Accelerated Electrification,gas_main,0.000000,0.000000
92,2025,Accelerated Electrification,gas_service,0.000000,0.000000
105,2025,Unmanaged Electrification,building_aggregate,2399.640525,2399.640525
106,2025,Unmanaged Electrification,gas_main,0.000000,0.000000
...,...,...,...,...,...
541,2050,Accelerated Electrification,gas_main,0.000000,0.000000
542,2050,Accelerated Electrification,gas_service,0.000000,0.000000
555,2050,Unmanaged Electrification,building_aggregate,0.000000,60011.101480
556,2050,Unmanaged Electrification,gas_main,0.000000,205014.538750


In [114]:
fig = px.bar(stranded_val, x="year", y="cum_val", color="asset_type",
             facet_col="scenario", facet_col_spacing=0.01, 
             barmode="stack",
             width = 700,
             category_orders={"scenario": sv_scenarios,
                            'asset_type':['building_aggregate','gas_service','gas_main', ],},
              color_discrete_map={'building_aggregate' : 'cornflowerblue',
                                'gas_main' : 'goldenrod', 
                                'gas_service' : 'palevioletred'},
              labels={'year':'', # remove 'year' from xaxis on each subplot
                    } 
            )

# update names for plotting
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
trace_dic = {'building_aggregate' : 'Building',
      'gas_main' : 'Gas Main', 
      'gas_service' : 'Gas Service'}
fig.for_each_trace(lambda t: t.update(name = trace_dic[t.name]))

# general formatting
fig.update_layout(bargap=0,
                  legend=dict(title_text='', x=0.92, y=0.98), # set legend location, properties
                  yaxis_title='Cumulative Stranded Value ($)',
                  plot_bgcolor='#f5f5f5',
                  )
fig.update_traces(marker_line_width=0)
fig.update_xaxes(tickangle= -90)
# fig.update_yaxes(showgrid=False)

fig.show()
fig.write_html(figure_outputs+'stranded_value.html')